In [136]:
import gurobipy as grb

In [137]:
import random
import pandas as pd
import time

In [138]:
def create_friend_groups(n,m):
    G = []
    for a1 in range(n):
        temp = [0]*n
        for a2 in range(n):
            if(a1//5 == a2//5):
                temp[a2] = 1
        G.append(temp)
    return G

def friend_groups_have_same_project(n,m):
  H = []
  for a in range(n):
    temp = [0]*m
    temp[a//5] = 1
    H.append(temp)
  return H

In [139]:
# Generate the social network matrix
def generate_G(n,m):
  G = []
  for i in range(n):
    randomlist = []
    for j in range(n):
      if i==j:
        randomlist.append(0)
      else:
        randomlist.append(random.choice([-1,0,1]))
    G.append(randomlist)
  return G

def generate_H(n,m):
  H = []
  for i in range(n):
    randomlist = []
    for j in range(m):
      rand_int = random.randint(0,1)
      randomlist.append(rand_int)
    H.append(randomlist)
  return H

def generate_P(n,m):
  P = []
  while(1):
    P = []
    temp2 = 0
    for i in range(m):
      # t1 is the upper bound for the number of teams that can work on that project
      # t3 is the upper bound for the team size of any of those teams
      (t1,t3) = (random.randint(1,3), random.randint(3,6))
      temp2 += t1*t3
      P.append((t1,t3))
    if(temp2 >= n):
      break
  return P

In [140]:
def check_constraints(y, L):
  lst_of_students=[]

  # every student gets exactly one project team
  flag_1 = 0
  for a in range(n):
    d = 0
    for i in range(m):
      for t in range(P[i][0]):
        if(y[(i,t,a)] == 1):
          d += 1
    if(d!=1):
      flag_1 += 1

  # L should represent bitwise AND of the y's
  flag_2 = 0
  for i in range(m):
    for t in range(P[i][0]):
      for a1 in range(n):
        for a2 in range(n):
          d1 = 10*int(y[(i,t,a1)])
          d2 = int(y[(i,t,a2)])
          d = int((d1 + d2)/11)
          if((L[(i,t,a1,a2)].x) != d):
            flag_2 += 1
  
  # team size is within upper bound
  flag_3 = 0
  for i in range(m):
    for t in range(P[i][0]):
      d = 0
      for a in range(n):
        if(y[(i,t,a)]== 1):
          d += 1
      if(d>P[i][1]):
        flag_3 += 1

  if(flag_1 == 0 and flag_2 == 0 and flag_3 == 0):
    return (True, 0)
  else:
    return (False, flag_1 + flag_2 + flag_3)

In [141]:
def our_solver(n, m, alpha, beta, G, H, P):
    start_time = time.perf_counter()
    GRB = grb.GRB

    model =grb.Model("Cluster")

    y_indices = []
    for i in range(m):
        for t in range(P[i][0]):
            for a in range(n):
                y_indices.append((i,t,a))


    y = model.addVars(y_indices,vtype=GRB.BINARY,name="y")

    l_indices = []
    for i in range(m):
        for t in range(P[i][0]):
            for a1 in range(n):
                for a2 in range(n):
                    l_indices.append((i,t,a1,a2))

    l = model.addVars(l_indices,vtype=GRB.BINARY,name="l")

    for a in range(n):
        model.addConstr( 1 == sum(y[(i,t,a)] for i in range (m) for t in range (P[i][0])) )

    for i in range(m):
        for t in range (P[i][0]):
            for a1 in range(n):
                for a2 in range(n):
                    model.addConstr(l[(i,t,a1,a2)]<=y[(i,t,a1)])
                    model.addConstr(l[(i,t,a1,a2)]<=y[(i,t,a2)])
                    model.addConstr(l[(i,t,a1,a2)]>= y[(i,t,a1)]+y[(i,t,a2)]-1)

    for i in range(m):
        for t in range (P[i][0]):
            model.addConstr( P[i][1] >= sum(y[(i,t,a)] for a in range(n)))

    model.setObjective(sum((y[(i,t,a)]*H[a][i]) for i in range(m) for t in range (P[i][0]) for a in range (n)) + 
    sum((l[(i,t,a1, a2)]*G[a1][a2]) for i in range(m) for t in range (P[i][0]) for a1 in range (n) for a2 in range(n) if (a1!=a2))
    ,GRB.MAXIMIZE)
    
    model.setParam("Timelimit", 600)
    model.setParam('OutputFlag', 0)
    model.optimize()

    time_taken = time.perf_counter() - start_time

    return (model.objVal , y, l , time_taken, model.MIPGap , model.status, model)

In [142]:
titles = ["test_num", "test_desc", "students", "projects", "time_taken(s)", "status", "obj_val", "constraints_satisfied", "num_constraints_broken"]
df = pd.DataFrame(columns = titles)
path = "gurobi_testing.csv"

In [143]:
# Test 1
n = 10
m = 3
alpha = 0.5
beta = 0.5
G = generate_G(n,m)
H = generate_H(n,m)
P = generate_P(n,m)
(objective_value, y_bar, l , time_taken , gap , status, model) = our_solver(n, m, alpha, beta, G, H, P)
y = {}
for key in y_bar:
  y[key] = int(y_bar[key].x)
(result, num_not_satisfied) = check_constraints(y , l);
print(f"Status = {status==2}, Objective value = {objective_value}, Time = {time_taken} seconds.")
print(f"Constraints satisfied = {result}, Number of constraints broken = {num_not_satisfied}.")


lst = [1, "Generic small example to see if testing structure is working", n, m, time_taken, status, objective_value, result, num_not_satisfied]
df.loc[len(df)] = lst


Changed value of parameter Timelimit to 600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Status = True, Objective value = 18.0, Time = 0.33909222200009026 seconds.
Constraints satisfied = True, Number of constraints broken = 0.


In [144]:
# Test 2
n = 15
m = 4
alpha = 0.5
beta = 0.5
G = generate_G(n,m)
H = generate_H(n,m)
P = generate_P(n,m)
(objective_value, y_bar, l , time_taken , gap , status, model) = our_solver(n, m, alpha, beta, G, H, P)
y = {}
for key in y_bar:
  y[key] = int(y_bar[key].x)
(result, num_not_satisfied) = check_constraints(y , l);
print(f"Status = {status==2}, Objective value = {objective_value}, Time = {time_taken} seconds.")
print(f"Constraints satisfied = {result}, Number of constraints broken = {num_not_satisfied}.")


lst = [1, "Generic small example to see if testing structure is working", n, m, time_taken, status, objective_value, result, num_not_satisfied]
df.loc[len(df)] = lst

Changed value of parameter Timelimit to 600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Status = True, Objective value = 34.0, Time = 0.6977077530000315 seconds.
Constraints satisfied = True, Number of constraints broken = 0.


In [145]:
df.to_csv(path)

In [146]:
# Test 3 - 9

'''
For all these, objective value should be = (n + 20m)/2 
'''
n_list = [15, 30, 50, 75, 100, 150, 200]
tester = 3
for n in n_list:
  m = n//5

  alpha = 0.5
  beta = 0.5

  G = create_friend_groups(n,m)

  H = friend_groups_have_same_project(n,m)

  P = [(1,5)]*m
  (objective_value, y_bar, l , time_taken , gap , status, model) = our_solver(n, m, alpha, beta, G, H, P)
  y = {}
  for key in y_bar:
    y[key] = int(y_bar[key].x)
  (result, num_not_satisfied) = check_constraints(y , l);
  print(f"Status = {status==2}, Objective value = {objective_value}, Time = {time_taken} seconds.")
  print(f"Constraints satisfied = {result}, Number of constraints broken = {num_not_satisfied}.")

  predicted_obj = (n+(20*m))
  print(objective_value == predicted_obj)
  print()

  msg = ""
  if (tester == 3):
    msg = "Easy eample where groups of friends have same project preferences fitting within the team size bounds (everything is multiples of 5)"
  else:
    msg = "Same structure as test 3 for increasing n and m"
  lst = [tester, msg, n, m, time_taken, status, objective_value, result, num_not_satisfied]
  df.loc[len(df)] = lst
  tester += 1


Changed value of parameter Timelimit to 600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Status = True, Objective value = 75.0, Time = 0.06390081200015629 seconds.
Constraints satisfied = True, Number of constraints broken = 0.
True

Changed value of parameter Timelimit to 600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Status = True, Objective value = 150.0, Time = 0.5178701779996118 seconds.
Constraints satisfied = True, Number of constraints broken = 0.
True

Changed value of parameter Timelimit to 600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Status = True, Objective value = 250.0, Time = 2.3879029879999507 seconds.
Constraints satisfied = True, Number of constraints broken = 0.
True

Changed value of parameter Timelimit to 600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Status = True, Objective value = 375.0, Time = 9.85201882399997 seconds.
Constraints satisfied = True, Number of constraints broken = 0.
True

Changed value of parameter Timelimit to 600.0
   P

In [147]:
df.to_csv(path)

In [148]:
# Test 10 - 16

'''
For all these, objective value should be = n + 8m
'''
n_list = [15, 30, 50, 75, 100, 150, 200]
tester = 10

for n in n_list:
  m = n//5

  alpha = 0.5
  beta = 0.5

  G = create_friend_groups(n,m)

  H = friend_groups_have_same_project(n,m)

  P = [(2,3)]*m

  (objective_value, y_bar, l , time_taken , gap , status, model) = our_solver(n, m, alpha, beta, G, H, P)
  y = {}
  for key in y_bar:
    y[key] = int(y_bar[key].x)
  (result, num_not_satisfied) = check_constraints(y , l);
  print(f"Status = {status==2}, Objective value = {objective_value}, Time = {time_taken} seconds.")
  print(f"Constraints satisfied = {result}, Number of constraints broken = {num_not_satisfied}.")

  predicted_obj = (n+(8*m))
  print(objective_value == predicted_obj)
  print()

  msg = ""
  if (tester == 10):
    msg = "Project preference and friend prefernece kept same as test 3, however number of sub teams in a project made to 2"
  else:
    msg = "Same structure as test 10 for increasing n and m"
  lst = [tester, msg, n, m, time_taken, status, objective_value, result, num_not_satisfied]
  df.loc[len(df)] = lst
  tester += 1


Changed value of parameter Timelimit to 600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Status = True, Objective value = 39.0, Time = 0.73935148999999 seconds.
Constraints satisfied = True, Number of constraints broken = 0.
True

Changed value of parameter Timelimit to 600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Status = True, Objective value = 78.0, Time = 58.16614836999997 seconds.
Constraints satisfied = True, Number of constraints broken = 0.
True

Changed value of parameter Timelimit to 600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Status = False, Objective value = 130.0, Time = 602.4967923389995 seconds.
Constraints satisfied = True, Number of constraints broken = 0.
True

Changed value of parameter Timelimit to 600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Status = False, Objective value = 195.0, Time = 324.47917090600004 seconds.
Constraints satisfied = True, Number of constraints broken = 0.
True

Changed value of parameter Timelimit to 600.0
   Pre

In [149]:
df.to_csv(path)

In [150]:
def create_modified_friend_groups(n,m):
  G = []
  for a1 in range(n):
    temp = [0]*n
    for a2 in range(n):
      if(a1//5 == a2//5):
        if(a1%5 in [0,1] and a2%5 in [0,1]):
          temp[a2] = 1
        elif(a1%5 in [2,3,4] and a2%5 in [2,3,4]):
          temp[a2] = 1
    G.append(temp)
  return G

In [151]:
# Test 17 - 23

n_list = [15, 30, 50, 75, 100, 150, 200]
tester = 17
for n in n_list:
  m = n//5

  alpha = 0.5
  beta = 0.5

  G = create_modified_friend_groups(n,m)

  H = friend_groups_have_same_project(n,m)

  P = [(2,3)]*m

  (objective_value, y_bar, l , time_taken , gap , status, model) = our_solver(n, m, alpha, beta, G, H, P)
  y = {}
  for key in y_bar:
    y[key] = int(y_bar[key].x)
  (result, num_not_satisfied) = check_constraints(y , l);
  print(f"Status = {status==2}, Objective value = {objective_value}, Time = {time_taken} seconds.")
  print(f"Constraints satisfied = {result}, Number of constraints broken = {num_not_satisfied}.")

  predicted_obj = (n+(8*m))
  print(objective_value == predicted_obj)
  print()

  msg = ""
  if (tester == 17):
    msg = "Project preference kept same as test 3, however number of sub teams in a project made to 2 and friendships also kept changed according to subteam division"
  else:
    msg = "Same structure as test 17 for increasing n and m"
  lst = [tester, msg, n, m, time_taken, status, objective_value, result, num_not_satisfied]
  df.loc[len(df)] = lst
  tester += 1

Changed value of parameter Timelimit to 600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Status = True, Objective value = 39.0, Time = 0.16348017500058631 seconds.
Constraints satisfied = True, Number of constraints broken = 0.
True

Changed value of parameter Timelimit to 600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Status = True, Objective value = 78.0, Time = 0.8385277530005624 seconds.
Constraints satisfied = True, Number of constraints broken = 0.
True

Changed value of parameter Timelimit to 600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Status = True, Objective value = 130.0, Time = 4.049404428999878 seconds.
Constraints satisfied = True, Number of constraints broken = 0.
True

Changed value of parameter Timelimit to 600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Status = True, Objective value = 195.0, Time = 15.600093609000396 seconds.
Constraints satisfied = True, Number of constraints broken = 0.
True

Changed value of parameter Timelimit to 600.0
   P

In [152]:
df.to_csv(path)